In [1]:
import torch
from transformers import AutoModelForCausalLM,AutoTokenizer
import time
import gc

/home/pa/miniconda3/envs/gen_ai/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model=AutoModelForCausalLM.from_pretrained("facebook/opt-125m")
print(model)


OPTForCausalLM(
  (model): OPTModel(
    (decoder): OPTDecoder(
      (embed_tokens): Embedding(50272, 768, padding_idx=1)
      (embed_positions): OPTLearnedPositionalEmbedding(2050, 768)
      (final_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (layers): ModuleList(
        (0-11): 12 x OPTDecoderLayer(
          (self_attn): OPTAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (activation_fn): ReLU()
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), ep

In [3]:
model=model.to('cpu')

In [4]:
torch.cuda.empty_cache()

In [5]:
question=input()
converstation=[
    {
        "role":"system",
        "content":"You are an assitant help user by solving their query"
    },
    {
        "role":"user",
        "content":f"query-{question}"
    }
]

In [5]:
question="hi"
tok=AutoTokenizer.from_pretrained("facebook/opt-125m")
# prompt=tok.apply_chat_template(converstation)
prompt=tok(question,return_tensors='pt').to('cuda')
# prompt.to('cpu'//)


In [6]:
# out=model.generate(**prompt,max_new_tokens=1000)
# print(tok.decode(out[0]))

In [7]:
input_ids=prompt['input_ids']
del prompt
torch.cuda.empty_cache()

In [8]:
input_ids.device

device(type='cuda', index=0)

In [18]:
model.model.decoder.embed_positions(torch.arange(x.shape[1]).unsqueeze(0).to('cuda')).device

NameError: name 'x' is not defined

In [9]:
model.model.decoder.embed_tokens.to('cuda')
x = model.model.decoder.embed_tokens(input_ids)
input_ids.to('cpu')
del input_ids
torch.cuda.empty_cache()
model.model.decoder.embed_tokens.to('cpu')
# del model.model.decoder.embed_tokens
torch.cuda.empty_cache()


In [10]:
model.model.decoder.embed_positions.to('cuda')
x = x + model.model.decoder.embed_positions(torch.arange(x.shape[1]).unsqueeze(0).to('cuda'))
model.model.decoder.embed_positions.to('cpu')
# del model.model.decoder.embed_positions
torch.cuda.empty_cache()

In [ ]:
def model_inference(question,tokens=250):
    for i in range(tokens):
        prompt=tok(question,return_tensors='pt').to('cuda')
        input_ids=prompt['input_ids']
        del prompt
        torch.cuda.empty_cache()
        model.model.decoder.embed_tokens.to('cuda')
        x = model.model.decoder.embed_tokens(input_ids)
        del input_ids
        torch.cuda.empty_cache()
        model.model.decoder.embed_tokens.to('cpu')
        # del model.model.decoder.embed_tokens
        torch.cuda.empty_cache()
        model.model.decoder.embed_positions.to('cuda')
        x = x + model.model.decoder.embed_positions(torch.arange(x.shape[1]).unsqueeze(0).to('cuda'))
        model.model.decoder.embed_positions.to('cpu')
        # del model.model.decoder.embed_positions
        torch.cuda.empty_cache()
        for i in model.model.decoder.layers:
            try:
                i.to('cuda')
                x=x.to('cuda')
                with torch.no_grad():
                    x=i(x)[0]
                i.to('cpu')
                torch.cuda.empty_cache()
                x=x.to('cpu')
                torch.cuda.empty_cache()
                # time.sleep(3)
            except:
                print(x)
                break
        model.lm_head.to('cuda')
        x=model.lm_head(x.to('cuda'))
        model.lm_head.to('cpu')
        torch.cuda.empty_cache()
        x1=x.argmax(-1)
        print(tok.decode(x1[0]),end=" ")
        question=question+" "+tok.decode(x1[0])
                

In [20]:
model_inference("How are you")

 How you should  How you should  To're be  How you should  To're be  You  .''. To.''.aned 

/pytorch/aten/src/ATen/native/cuda/Indexing.cu:1553: indexSelectLargeIndex: block: [26,0,0], thread: [64,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
/pytorch/aten/src/ATen/native/cuda/Indexing.cu:1553: indexSelectLargeIndex: block: [26,0,0], thread: [65,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
/pytorch/aten/src/ATen/native/cuda/Indexing.cu:1553: indexSelectLargeIndex: block: [26,0,0], thread: [66,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
/pytorch/aten/src/ATen/native/cuda/Indexing.cu:1553: indexSelectLargeIndex: block: [26,0,0], thread: [67,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
/pytorch/aten/src/ATen/native/cuda/Indexing.cu:1553: indexSelectLargeIndex: block: [26,0,0], thread: [68,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
/pytorch/aten/src/ATen/native/cuda/Indexing.cu:1553: indexSelectLargeIndex: block: [26,0,0], thread: [69,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
/pytorch/aten/src/ATen/native/cuda/Indexing.cu:1553:

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [11]:
print()
cnt=0
for i in model.model.decoder.layers:
    cnt+=1
    print(cnt)
    try:
        i.to('cuda')
        x.to('cuda')
        with torch.no_grad():
            x=i(x)[0]
        i.to('cpu')
        torch.cuda.empty_cache()
        x.to('cpu')
        torch.cuda.empty_cache()
        # time.sleep(3)
    except:
        print(cnt)
        print(x)
        break


1
2
3
4
5
6
7
8
9
10
11
12


In [12]:
model.lm_head.to('cuda')
x=model.lm_head(x.to('cuda'))
model.lm_head.to('cpu')
torch.cuda.empty_cache()

In [13]:
x1=x.argmax(-1)

In [14]:
tok.decode(x1[0])

' hi'